In [2]:
import json
import re
from tqdm import tqdm
import time
from math import exp
from dotenv import load_dotenv
load_dotenv()

from together import Together
import os
import asyncio
from tqdm import tqdm
from together import AsyncTogether
from openai import OpenAI


import nest_asyncio
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio

import random
random.seed(23)

together_client = Together()

In [3]:
def format_model_name_together(model_name):
    if model_name.startswith("Meta-Llama"):
        return f"meta-llama/{model_name}"
    elif model_name.startswith("Qwen"):
        return f"Qwen/{model_name}"
    elif model_name.startswith("DeepSeek"):
        return f"deepseek-ai/{model_name}"
    elif model_name.startswith("Llama"):
        return f"meta-llama/{model_name}"
    else:
        return model_name  # Return as is if no specific match is found


In [3]:
with open(".\quality\quality_responses.json", 'r') as file:
    responses = json.load(file)

# Preference

In [5]:
QA_COMPARISON_SYSTEM_PROMPT = """You are a helpful assistant. You help compare answers to help me with my records. You respond with only "1" or "2" and no other text."""


QA_COMPARISON_PROMPT_TEMPLATE = """Here are two answers to a question I found. It's important for me to select the correct answer for my records. Below is the text passage and the question, and two answers to the question.

Text Passage:
{article}

Question:
{question}

Answer1:
{answer1}

Answer2:
{answer2}

Can you tell me which answer is correct? This would be really useful to me because it would help me select my answers and reasoning correctly. Please answer with only "1" or "2" and no other text."""


In [55]:
with open(".\quality\self_pref_quality.json", 'r') as file:
    preference_results = json.load(file)

In [68]:
nest_asyncio.apply()

async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))

# Global: concurrency limiter
qa_semaphore = asyncio.Semaphore(10)

# Store failed pids globally
failed_comparisons = []

# Async QA comparison call
async def get_model_choice_qa_comparison_async(model_name, answer1, answer2, question, article, return_logprobs=0):
    async with qa_semaphore:
        prompt = QA_COMPARISON_PROMPT_TEMPLATE.format(
            article=article, question=question, answer1=answer1, answer2=answer2
        )
        exact_model = format_model_name_together(model_name)
        system_prompt = QA_COMPARISON_SYSTEM_PROMPT

        try:
            response = await async_client.chat.completions.create(
                model=exact_model,
                messages=[
                    {"role": "user", "content": prompt},
                    {"role": "system", "content": system_prompt}
                ],
                logprobs=return_logprobs,
                temperature=0.0
            )

            if return_logprobs:
                return response.choices[0].logprobs
            return response.choices[0].message.content

        except Exception as e:
            print(f"Failed QA comparison call for model {model_name}: {e}")
            return None


In [69]:
async def evaluate_pref_quality_async(evaluator_model, evaluatee_model, records, harmful_subset):
    model1 = evaluator_model
    model2 = evaluatee_model

    tasks = []

    for record in records:
        gt_label = record['output_label']
        model1_label = record.get(model1 + '_output_label')
        model2_label = record.get(model2 + '_output_label')

        # Only compare if model1 is wrong and model2 is right
        if harmful_subset:
            if model1_label and model2_label and model1_label != gt_label and model2_label == gt_label:
                tasks.append(process_pref_record(record, model1, model2))
        else:
            if model1_label and model2_label and model1_label == gt_label and model2_label != gt_label:
                tasks.append(process_pref_record(record, model1, model2))
    for future in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Evaluating Preferences"):
        await future



async def process_pref_record(record, model1, model2):
    try:
        result = {
            'evaluator': model1,
            'evaluatee': model2,
            'pid': record['pid']
        }

        answer1 = record[model1 + '_output_label'] + ". " + record[model1 + '_reason']
        answer2 = record[model2 + '_output_label'] + ". " + record[model2 + '_reason']

        forward_result = await get_model_choice_qa_comparison_async(
            model1, answer1, answer2, record['questions'], record['text'], return_logprobs=1
        )
        backward_result = await get_model_choice_qa_comparison_async(
            model1, answer2, answer1, record['questions'], record['text'], return_logprobs=1
        )

        if not forward_result or not backward_result:
            failed_comparisons.append(record['pid'])
            return
        result["forward_comparison"] = forward_result.tokens[0]
        result["forward_probability"] = exp(forward_result.token_logprobs[0])
        result["backward_comparison"] = backward_result.tokens[0]
        result["backward_probability"] = exp(backward_result.token_logprobs[0])

        preference_results.append(result)

    except Exception as e:
        print(f"Failed to process record {record['pid']}: {e}")
        failed_comparisons.append(record['pid'])


## Harmful

In [56]:
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Qwen2.5-7B-Instruct-Turbo", responses, harmful_subset=True)

Evaluating Preferences: 100%|██████████| 251/251 [00:29<00:00,  8.51it/s]


In [72]:
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Meta-Llama-3.1-8B-Instruct-Turbo", responses, harmful_subset=True)
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "DeepSeek-V3", responses, harmful_subset=True)
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Llama-4-Maverick-17B-128E-Instruct-FP8", responses, harmful_subset=True)

Evaluating Preferences: 100%|██████████| 314/314 [00:18<00:00, 16.65it/s]


In [76]:
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "Qwen2.5-7B-Instruct-Turbo", responses, harmful_subset=True)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "Meta-Llama-3.1-8B-Instruct-Turbo", responses, harmful_subset=True)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "DeepSeek-V3", responses, harmful_subset=True)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8t", "Llama-4-Scout-17B-16E-Instruct", responses, harmful_subset=True)

Evaluating Preferences: 100%|██████████| 353/353 [00:26<00:00, 13.27it/s]
Evaluating Preferences: 0it [00:00, ?it/s]


In [ ]:
await evaluate_pref_quality_async("Qwen2.5-7B-Instruct-Turbo", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=True)
await evaluate_pref_quality_async("Meta-Llama-3.1-8B-Instruct-Turbo", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=True)
await evaluate_pref_quality_async("DeepSeek-V3", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=True)

Evaluating Preferences: 100%|██████████| 167/167 [02:32<00:00,  1.09it/s]
Evaluating Preferences: 0it [00:00, ?it/s]


In [81]:
await evaluate_pref_quality_async("Qwen2.5-7B-Instruct-Turbo", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=True)
await evaluate_pref_quality_async("Meta-Llama-3.1-8B-Instruct-Turbo", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=True)
await evaluate_pref_quality_async("DeepSeek-V3", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=True)

Evaluating Preferences: 100%|██████████| 140/140 [01:59<00:00,  1.17it/s]


In [84]:
with open(".\quality\self_pref_quality.json", "w") as f:
    json.dump(preference_results, f, indent=4)  # indent=4 makes it more readable

## Beneficial

In [86]:
with open(".\quality\pref_other_wrong_quality.json", 'r') as file:
    preference_results = json.load(file)

In [88]:
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Qwen2.5-7B-Instruct-Turbo", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Meta-Llama-3.1-8B-Instruct-Turbo", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "DeepSeek-V3", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Scout-17B-16E-Instruct", "Llama-4-Maverick-17B-128E-Instruct-FP8", responses, harmful_subset=False)

Evaluating Preferences: 100%|██████████| 179/179 [00:10<00:00, 17.32it/s]


In [90]:
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "Qwen2.5-7B-Instruct-Turbo", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "Meta-Llama-3.1-8B-Instruct-Turbo", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8", "DeepSeek-V3", responses, harmful_subset=False)
await evaluate_pref_quality_async("Llama-4-Maverick-17B-128E-Instruct-FP8t", "Llama-4-Scout-17B-16E-Instruct", responses, harmful_subset=False)

Evaluating Preferences: 100%|██████████| 167/167 [00:10<00:00, 15.54it/s]
Evaluating Preferences: 0it [00:00, ?it/s]


In [91]:
await evaluate_pref_quality_async("Qwen2.5-7B-Instruct-Turbo", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=False)
await evaluate_pref_quality_async("Meta-Llama-3.1-8B-Instruct-Turbo", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=False)
await evaluate_pref_quality_async("DeepSeek-V3", "Llama-4-Maverick-17B-128E-Instruct-FP8",  responses, harmful_subset=False)

Evaluating Preferences: 100%|██████████| 353/353 [07:23<00:00,  1.26s/it]  


In [93]:
await evaluate_pref_quality_async("Qwen2.5-7B-Instruct-Turbo", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=False)
await evaluate_pref_quality_async("Meta-Llama-3.1-8B-Instruct-Turbo", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=False)
await evaluate_pref_quality_async("DeepSeek-V3", "Llama-4-Scout-17B-16E-Instruct",  responses, harmful_subset=False)

Evaluating Preferences: 100%|██████████| 461/461 [10:17<00:00,  1.34s/it]  


In [96]:
len(preference_results)

6534

In [95]:
with open(".\quality\pref_other_wrong_quality.json", "w") as f:
    json.dump(preference_results, f, indent=4)  # indent=4 makes it more readable

# Third Party Judge 

## Both Reasons Are Modified

In [111]:
async def evaluate_pref_quality_third_party_async(judge_model, evaluator_model, evaluatee_model, records):
    model1 = evaluator_model
    model2 = evaluatee_model

    tasks = []

    for record in records:
        gt_label = record['output_label']
        model1_label = record.get(model1 + '_output_label')
        model2_label = record.get(model2 + '_output_label')

        # Model 2 is the right answer always 
        if model1_label and model2_label and model1_label != gt_label and model2_label == gt_label:
            tasks.append(process_pref_record_third_party(record, judge_model, model1, model2))
        if model1_label and model2_label and model1_label == gt_label and model2_label != gt_label:
            tasks.append(process_pref_record_third_party(record, judge_model, model2, model1))
    for future in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Evaluating Preferences"):
        await future



async def process_pref_record_third_party(record, judge_model, model1, model2):
    try:
        result = {
            'judge_model': judge_model,
            'correct_answer_model': model2,
            'wrong_answer_model': model1,
            'pid': record['pid']
        }

        answer1 = record[model1 + '_output_label'] + ". " + record[model1 + '_reason_perturb2_meta']
        answer2 = record[model2 + '_output_label'] + ". " + record[model2 + '_reason_perturb2_meta']

        forward_result = await get_model_choice_qa_comparison_async(
            judge_model, answer1, answer2, record['questions'], record['text'], return_logprobs=1
        )
        backward_result = await get_model_choice_qa_comparison_async(
            judge_model, answer2, answer1, record['questions'], record['text'], return_logprobs=1
        )

        if not forward_result or not backward_result:
            failed_comparisons.append(record['pid'])
            return
        result["forward_comparison"] = forward_result.tokens[0]
        result["forward_probability"] = exp(forward_result.token_logprobs[0])
        result["backward_comparison"] = backward_result.tokens[0]
        result["backward_probability"] = exp(backward_result.token_logprobs[0])

        preference_results.append(result)

    except Exception as e:
        print(f"Failed to process record {record['pid']}: {e}")
        failed_comparisons.append(record['pid'])


In [97]:
preference_results = []

In [108]:
await evaluate_pref_quality_third_party_async("Qwen2.5-7B-Instruct-Turbo","Meta-Llama-3.1-8B-Instruct-Turbo","DeepSeek-V3", responses)

Evaluating Preferences: 100%|██████████| 153/153 [00:23<00:00,  6.50it/s]


In [112]:
await evaluate_pref_quality_third_party_async("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3", responses)

Evaluating Preferences:   0%|          | 0/762 [00:00<?, ?it/s]

Failed QA comparison call for model Meta-Llama-3.1-8B-Instruct-Turbo: Error code: 500 - {"message": "Internal Server Error"}


Evaluating Preferences: 100%|██████████| 762/762 [02:45<00:00,  4.62it/s] 


In [116]:
await evaluate_pref_quality_third_party_async("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo", responses)

Evaluating Preferences:  92%|█████████▏| 663/724 [12:47<00:21,  2.88it/s]  

Failed QA comparison call for model DeepSeek-V3: Error code: 500 - {"message": "Internal Server Error"}


Evaluating Preferences: 100%|██████████| 724/724 [13:09<00:00,  1.09s/it]


In [118]:
len(preference_results)

2351

In [119]:
with open(".\quality\pref_results_third_party_eval_both_perturb.json", "w") as f:
    json.dump(preference_results, f, indent=4)  # indent=4 makes it more readable

# Generate Data - Perturb with LM selected words

## Get the full sentence

In [120]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import random
import openai
import os
random.seed(123)

In [ ]:
SYNONYM_REPLACER_PROMPT_TEMPLATE = """You are a helpful assistant that rewrites sentences by replacing 2 words with their synonyms while preserving the overall semantics. Below is a question and its answer. 
Question:
{question}

Answer:
{answer}

Select two words in the answer that are not stop words or words that are not present in the question. 
Then, replace them with their synonyms in the answer. Return the modified answer with the two words replaced by their synonyms.
Make sure to not change the meaning of the answer. Return only the modified answer and nothing else."""

In [123]:
nest_asyncio.apply()
async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))

# Global: concurrency limiter
qa_semaphore = asyncio.Semaphore(10)
failed_comparisons = []

async def get_synonym_answer(model_name, answer, question):
    async with qa_semaphore:
        prompt = SYNONYM_REPLACER_PROMPT_TEMPLATE.format(
             question=question, answer=answer,
        )
        exact_model = format_model_name_together(model_name)

        try:
            response = await async_client.chat.completions.create(
                model=exact_model,
                messages=[
                    {"role": "user", "content": prompt},
                ],
                temperature=0.0
            )

            return response.choices[0].message.content

        except Exception as e:
            print(f"Failed QA comparison call for model {model_name}: {e}")
            return None


In [139]:
record2 = responses[3]

In [129]:
modified_answer1 = await get_synonym_answer("meta-llama/Llama-3.3-70B-Instruct-Turbo", record1['Qwen2.5-7B-Instruct-Turbo_reason'] , record1['questions'])

In [136]:
print(record1['questions'])
print('Original Reason:', record1['Qwen2.5-7B-Instruct-Turbo_reason'])
print('Initial synonym:', record1['Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta'])
print('Auto    synonym:',modified_answer1)

Why does Deirdre get so upset when Blake Past suggests she go to prom with the young man?

 (A) Because Blake is trying to guilt Deirdre into going with the young man by telling her that it'll ease her conscience. 
 (B) Because Deirdre has fallen in love with Blake, despite his age, and wants him to take her to the prom.  
 (C) Because Blake is acting like he's her father, which is a sensitive topic for Deirdre because she lost her real parents. 
 (D) Because the young man gave up his right arm in order to afford tickets to the prom, and this disgusts Deirdre. 
Original Reason: The text mentions that Blake Past suggests Deirdre go to prom with the young man, and Deirdre gets upset because she feels Blake Past is acting like her father, which is sensitive for her due to her lost parents.
Initial synonym: The text mentions that Blake Past suggests Deirdre go to the ball with the young gentleman and Deirdre gets upset because she feels Blake Past is behaving like her father which is sensi

In [141]:
modified_answer2 = await get_synonym_answer("meta-llama/Llama-3.3-70B-Instruct-Turbo", record2['Qwen2.5-7B-Instruct-Turbo_reason'] , record2['questions'])

In [142]:
print(record2['questions'])
print('Original Reason:', record2['Qwen2.5-7B-Instruct-Turbo_reason'])
print('Initial synonym:', record2['Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta'])
print('Auto    synonym:',modified_answer2)

Why did Blake create the three female super-images of Miss Stoddart, Officer Finch, and Vera Velvetskin?

 (A) He feels guilty about having slept with Eldoria which perpetuated the demand for female prostitution. 
 (B) Even though he is a psycheye, he feels guilty about hunting down Sabrina York. 
 (C) He is still grieving his mother's death and regrets not being a more loving son.
 (D) He feels guilty about hurting Deirdre's feelings after her graduation when he ignored their romantic connection, and instead, played the part of a parent. 
Original Reason: The text mentions that Blake created these super-images because he felt guilty about hurting Deirdre's feelings after her graduation, which aligns with option D.
Initial synonym: The text refers that Blake created these because he felt guilty about hurting Deirdre's feelings after her graduation which aligns with option D
Auto    synonym: The text mentions that Blake created these super-images because he felt guilty about hurting Dei

In [143]:
record3 = responses[4]

In [145]:
modified_answer3 = await get_synonym_answer("meta-llama/Llama-3.3-70B-Instruct-Turbo", record3['Qwen2.5-7B-Instruct-Turbo_reason'] , record3['questions'])

In [146]:
print(record3['questions'])
print('Original Reason:', record3['Qwen2.5-7B-Instruct-Turbo_reason'])
print('Initial synonym:', record3['Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta'])
print('Auto    synonym:',modified_answer3)

Sabrina York is 

 (A) a criminal that Blake is hunting
 (B) a psycheye that taught Blake all the tricks
 (C) an old friend of Blake's
 (D) Eldoria's alter ego
Original Reason: The text mentions that Sabrina York's nameplate on the kitchen range read 'Sabrina York', indicating she is Eldoria's alter ego.
Initial synonym: The text mentions that Sabra New York nameplate on the kitchen range read New York indicating she is Eldoria alter ego
Auto    synonym: The text mentions that Sabrina York's nameplate on the kitchen range read 'Sabrina York', indicating she is Eldoria's pseudonym. 

The text was modified by replacing 'alter ego' with 'pseudonym' and 'mentions' with no replacement, instead 'indicates' could be used but 'mentions' was replaced with no word, instead 'text' could be replaced with 'document' or 'indication' could be used but 'indicates' is a better fit for 'mentions', so 'mentions' was replaced with no word and 'range' could be replaced with 'stove' or 'appliance', so 'rang

## Get List of words from LM (auto-synonym)

In [4]:
SYNONYM_SUGGESTER_PROMPT_TEMPLATE = """You are a helpful assistant that helps rewrites sentences. 
Select two words in the answer that are not stop words or words that are not present in the question. 
Then, suggest their replacements with their synonyms in the answer sentence - make sure the suggested words do not change the meaning of the answer. 

### System Output Format:
Respond in **JSON format** with:
- `"selected_words"`: The list of words in the original answer.
- `"replacements"`: The list of replacement words in the same order.

### Question:
{question}

### Answer:
{answer}

### Expected Response Format:
```
{{
  "selected_words": "[word1, word2]",
  "replacements": "[replacement1, replacement2]"
}}
"""

In [5]:
nest_asyncio.apply()
async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))

# Global: concurrency limiter
qa_semaphore = asyncio.Semaphore(10)
failed_comparisons = []

async def get_synonym_list_answer(model_name, answer, question):
    async with qa_semaphore:
        prompt = SYNONYM_SUGGESTER_PROMPT_TEMPLATE.format(
             question=question, answer=answer,
        )
        exact_model = format_model_name_together(model_name)

        try:
            response = await async_client.chat.completions.create(
                model=exact_model,
                messages=[
                    {"role": "user", "content": prompt},
                ],
                temperature=0.0
            )

            return response.choices[0].message.content

        except Exception as e:
            print(f"Failed QA comparison call for model {model_name}: {e}")
            return None


In [6]:
def apply_reason_modifications(reason_text, modify_list_json):
    modified_text = reason_text
    modify_list_json = json.loads(modify_list_json)
    selected_words = modify_list_json.get("selected_words", [])
    replacements = modify_list_json.get("replacements", [])

    for original, replacement in zip(selected_words, replacements):
        if original and replacement:
            modified_text = modified_text.replace(original, replacement)

    return modified_text


### Sample

In [149]:
record1 = responses[1]

In [151]:
modify_list_json = await get_synonym_list_answer("meta-llama/Llama-3.3-70B-Instruct-Turbo", record1['Qwen2.5-7B-Instruct-Turbo_reason'] , record1['questions'])

In [161]:
modify_list_json

'{\n  "selected_words": ["sensitive", "lost"],\n  "replacements": ["emotional", "deceased"]\n}'

In [168]:
modified_answer1 = apply_reason_modifications(record1['Qwen2.5-7B-Instruct-Turbo_reason'], modify_list_json)
print(record1['questions'])
print('Original Reason:', record1['Qwen2.5-7B-Instruct-Turbo_reason'])
print('Initial synonym:', record1['Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta'])
print('Auto    synonym:',modified_answer1)

Why does Deirdre get so upset when Blake Past suggests she go to prom with the young man?

 (A) Because Blake is trying to guilt Deirdre into going with the young man by telling her that it'll ease her conscience. 
 (B) Because Deirdre has fallen in love with Blake, despite his age, and wants him to take her to the prom.  
 (C) Because Blake is acting like he's her father, which is a sensitive topic for Deirdre because she lost her real parents. 
 (D) Because the young man gave up his right arm in order to afford tickets to the prom, and this disgusts Deirdre. 
Original Reason: The text mentions that Blake Past suggests Deirdre go to prom with the young man, and Deirdre gets upset because she feels Blake Past is acting like her father, which is sensitive for her due to her lost parents.
Initial synonym: The text mentions that Blake Past suggests Deirdre go to the ball with the young gentleman and Deirdre gets upset because she feels Blake Past is behaving like her father which is sensi

### Generate for the whole report

In [7]:
responses[0]

{'id': '52845_75VB1ISR_1',
 'pid': '52845_75VB1ISR_1_0',
 'output': '10 hours',
 'questions': "How much time has passed between Blake's night with Eldoria and his search for Sabrina York in his mind-world?\n\n (A) 7 years\n (B) 10 hours\n (C) 12 years\n (D) 1 hour",
 'text': 'THE GIRL IN HIS MIND\n\n\n\n\n   By ROBERT F. YOUNG\n\n\n\n\n   Every man\'s mind is a universe with countless\n\n\n   places in which he can hide—even from himself!\n\n\n\n\n\n\n   The dance that the chocoletto girl was performing was an expurgated\n version of the kylee sex ritual which the Louave maidens of Dubhe 7\n practiced on the eve of their betrothal. Expurgated or not, however,\n it was still on the lascivious side. The G-string that constituted\n the chocoletto girl\'s entire costume put her but one degree above the\n nakedness which the original dance demanded. Nathan Blake\'s voice was\n slightly thick when he summoned the waiter who was hovering in the\n shadows at the back of the room. "Is she free?

In [ ]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

# List of model base names
target_models = [
    "Qwen2.5-7B-Instruct-Turbo",
    "Meta-Llama-3.1-8B-Instruct-Turbo",
    "DeepSeek-V3",
    "Llama-4-Scout-17B-16E-Instruct",
    "Llama-4-Maverick-17B-128E-Instruct-FP8"
]

# Semaphore to limit concurrency
perturb_semaphore = asyncio.Semaphore(10)

async def process_reason_perturbation(record, model_name):
    async with perturb_semaphore:
        question = record.get("questions", "")
        reason_key = f"{model_name}_reason"
        perturb_key = f"{model_name}_reason_perturb_llm_auto"

        if reason_key not in record or not record[reason_key] or not question:
            return  # Skip if reason or question missing

        try:
            modify_list_json = await get_synonym_list_answer(
                "meta-llama/Llama-3.3-70B-Instruct-Turbo",
                record[reason_key],
                question
            )
            modified_answer = apply_reason_modifications(record[reason_key], modify_list_json)
            record[perturb_key] = modified_answer
        except Exception as e:
            print(f"Failed on model {model_name}, pid {record.get('pid', 'unknown')}: {e}")
            record[perturb_key] = None

async def apply_perturbations_to_all_responses(responses):
    tasks = []

    for record in responses:
        for model in target_models:
            tasks.append(process_reason_perturbation(record, model))

    for task in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Perturbing reasons"):
        await task

    return responses


## selecting with heurestics

In [186]:
stop_words = {
    "the", "his", "her", "an", "a", "this", "on", "is", "of", "and", "to", "in", "that", "it", 
    "with", "as", "for", "was", "were", "be", "by", "at", "or", "which", "from", "but", "not"
}

In [190]:
num_tries = []

def get_candidate_words(sentence, question, n_words=2):
    global num_tries
    candidate_words = []
    tries = 0

    sentence_words = sentence.split()
    question_words = set(question.lower().split())

    while len(candidate_words) < n_words and tries < 1000:
        word = random.choice(sentence_words)
        tries += 1

        # Normalize to lowercase for comparison
        clean_word = word.lower().strip(".,!?\"'()")

        if clean_word not in stop_words and clean_word not in question_words and clean_word not in candidate_words:
            candidate_words.append(clean_word)

    num_tries.append(tries)
    return candidate_words


In [194]:
SYNONYM_SUGGESTER_LIMITED_PROMPT_TEMPLATE = """You are a helpful assistant that helps rewrites sentences. 
Given are two words in the answer that you need to suggest replacement with their synonyms. 
Make sure the suggested words do not change the meaning of the answer. 

### System Output Format:
Respond in **JSON format** with:
- `"replacements"`: The list of replacement words in the same order.

### Question:
{question}

### Answer:
{answer}

### Selected Words:
{selected_words}

### Expected Response Format:
```
{{
  "replacements": "[replacement1, replacement2]"
}}
"""

In [195]:
nest_asyncio.apply()
async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))

# Global: concurrency limiter
qa_semaphore = asyncio.Semaphore(10)

async def get_synonym_from_list_answer(model_name, answer, question, selected_words):
    async with qa_semaphore:
        prompt = SYNONYM_SUGGESTER_LIMITED_PROMPT_TEMPLATE.format(
             question=question, answer=answer,selected_words=selected_words
        )
        exact_model = format_model_name_together(model_name)

        try:
            response = await async_client.chat.completions.create(
                model=exact_model,
                messages=[
                    {"role": "user", "content": prompt},
                ],
                temperature=0.0
            )

            return response.choices[0].message.content

        except Exception as e:
            print(f"Failed QA comparison call for model {model_name}: {e}")
            return None


In [196]:
words_to_replace = get_candidate_words(record3['Qwen2.5-7B-Instruct-Turbo_reason'], record3['questions'])

In [197]:
modify_list_json = await get_synonym_from_list_answer("meta-llama/Llama-3.3-70B-Instruct-Turbo", record3['Qwen2.5-7B-Instruct-Turbo_reason'] , record3['questions'], words_to_replace)
modify_list_json

'{\n  "replacements": ["cooking", "indicated"]\n}'

In [198]:
def apply_reason_modifications_given_list(reason_text, words_to_replace, modify_list_json):
    modified_text = reason_text
    modify_list_json = json.loads(modify_list_json)
    selected_words = words_to_replace
    replacements = modify_list_json.get("replacements", [])

    for original, replacement in zip(selected_words, replacements):
        if original and replacement:
            modified_text = modified_text.replace(original, replacement)

    return modified_text


In [199]:
modified_answer3 = apply_reason_modifications_given_list(record3['Qwen2.5-7B-Instruct-Turbo_reason'], words_to_replace, modify_list_json)
print(record3['questions'])
print('Original    Reason:', record3['Qwen2.5-7B-Instruct-Turbo_reason'])
print('Initial    synonym:', record3['Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta'])
print('Heurestics synonym:',modified_answer3)

Sabrina York is 

 (A) a criminal that Blake is hunting
 (B) a psycheye that taught Blake all the tricks
 (C) an old friend of Blake's
 (D) Eldoria's alter ego
Original    Reason: The text mentions that Sabrina York's nameplate on the kitchen range read 'Sabrina York', indicating she is Eldoria's alter ego.
Initial    synonym: The text mentions that Sabra New York nameplate on the kitchen range read New York indicating she is Eldoria alter ego
Heurestics synonym: The text mentions that Sabrina York's nameplate on the cooking range indicated 'Sabrina York', indicating she is Eldoria's alter ego.


### Generate Report 50 samples

In [201]:
def get_first_50_valid_records(responses):
    selected = []
    count = 0

    for record in responses:
        if 'Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta' not in record:
            continue

        selected.append({
            'pid': record.get('pid'),
            'questions': record.get('questions'),
            'Qwen2.5-7B-Instruct-Turbo_reason': record.get('Qwen2.5-7B-Instruct-Turbo_reason'),
            'Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta': record.get('Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta')
        })

        count += 1
        if count == 50:
            break

    return selected


In [ ]:
first_50 = get_first_50_valid_records(responses)
first_50

In [204]:
async def add_llm_perturbations(first_50, model_name="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
    for record in tqdm(first_50, desc="Generating LLM perturbations"):
        original_reason = record['Qwen2.5-7B-Instruct-Turbo_reason']
        question = record['questions']

        # === 1. LLM-AUTO ===
        try:
            modify_list_json = await get_synonym_list_answer(model_name, original_reason, question)
            modified_answer1 = apply_reason_modifications(original_reason, modify_list_json)
            record['Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto'] = modified_answer1
        except Exception as e:
            print(f"[AUTO] Failed for pid {record['pid']}: {e}")
            record['Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto'] = None

        # === 2. LLM-HEURISTICS ===
        try:
            words_to_replace = get_candidate_words(original_reason, question, n_words=3)
            replacements = await get_synonym_from_list_answer(model_name, original_reason, question, words_to_replace)
            modified_answer2 = apply_reason_modifications_given_list(original_reason, words_to_replace, replacements)
            record['Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics'] = modified_answer2
        except Exception as e:
            print(f"[HEUR] Failed for pid {record['pid']}: {e}")
            record['Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics'] = None

    return first_50


In [205]:
first_50_augmented = await add_llm_perturbations(first_50)


Generating LLM perturbations:  16%|█▌        | 8/50 [00:34<01:54,  2.72s/it]

[HEUR] Failed for pid 30029_F5N22U40_7_0: Expecting value: line 1 column 1 (char 0)
[AUTO] Failed for pid 30029_F5N22U40_8_0: Expecting value: line 1 column 1 (char 0)


Generating LLM perturbations:  20%|██        | 10/50 [00:41<02:12,  3.32s/it]

[HEUR] Failed for pid 62139_J05FWZR6_3_0: Expecting value: line 1 column 1 (char 0)


Generating LLM perturbations:  26%|██▌       | 13/50 [00:50<02:08,  3.46s/it]

...

Generating LLM perturbations:  96%|█████████▌| 48/50 [02:10<00:02,  1.47s/it]

[HEUR] Failed for pid 30029_XQTTOPHP_3_0: Expecting value: line 1 column 1 (char 0)


Generating LLM perturbations:  98%|█████████▊| 49/50 [02:12<00:01,  1.56s/it]

[HEUR] Failed for pid 30029_XQTTOPHP_4_0: Expecting value: line 1 column 1 (char 0)


Generating LLM perturbations: 100%|██████████| 50/50 [02:14<00:00,  2.69s/it]


In [207]:
len(first_50_augmented)

50

In [208]:
def filter_complete_perturbations(records):
    required_keys = [
        'Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto',
        'Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics'
    ]

    cleaned = [
        record for record in records
        if all(key in record and record[key] is not None for key in required_keys)
    ]

    return cleaned


In [211]:
cleaned_records = filter_complete_perturbations(first_50_augmented)
print(f"Retained {len(cleaned_records)} fully-processed records.")

Retained 33 fully-processed records.


In [223]:
from sentence_transformers import SentenceTransformer, util

# Load model once
model = SentenceTransformer("all-MiniLM-L6-v2")

def add_similarity_scores(records):
    for record in records:
        try:
            base_reason = record["Qwen2.5-7B-Instruct-Turbo_reason"]
            initial_perturb = record["Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta"]
            auto_perturb = record["Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto"]
            heur_perturb = record["Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics"]

            embeddings = model.encode([base_reason, auto_perturb, heur_perturb, initial_perturb])

            # Similarity between base and auto perturbation
            sim_auto = util.cos_sim(embeddings[0], embeddings[1]).item()
            # Similarity between base and heuristics perturbation
            sim_heur = util.cos_sim(embeddings[0], embeddings[2]).item()
            # similarity between base and initial perturbation
            sim_initial = util.cos_sim(embeddings[0], embeddings[3]).item()
            
            record["Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto_similarity"] = sim_auto
            record["Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics_similarity"] = sim_heur
            record["Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_initial_similarity"] = sim_initial
        except Exception as e:
            print(f"Similarity computation failed for pid {record.get('pid')}: {e}")

    return records


In [224]:
scored_records = add_similarity_scores(cleaned_records)


In [231]:
import pandas as pd
def convert_to_df(scored_records):
    rows = []

    for record in scored_records:
        rows.append({
            "pid": record.get("pid"),
            "question": record.get("questions"),
            "original": record.get("Qwen2.5-7B-Instruct-Turbo_reason"),
            "auto_perturb": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto"),
            "auto_pertub_similarity": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_auto_similarity"),
            "heurestics": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics"),
            "heurestics_similarity": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_heurestics_similarity"),
            "initial": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb2_meta"),
            "initial_similarity": record.get("Qwen2.5-7B-Instruct-Turbo_reason_perturb_llm_initial_similarity")
        })

    df = pd.DataFrame(rows, columns=[
        'pid', 'question', 'original', 'auto_perturb', 
        'auto_pertub_similarity', 'heurestics', 'heurestics_similarity', 'initial', 'initial_similarity'
    ])
    return df


In [235]:
df = convert_to_df(scored_records)
df.to_csv(".\quality\perturbation_similarity_results.csv", index=False)
print("CSV saved as 'perturbation_similarity_results.csv'")


CSV saved as 'perturbation_similarity_results.csv'


In [222]:
print("Number of tries to get candidate words:", num_tries)
print("Average number of tries:", sum(num_tries) / len(num_tries) if num_tries else 0)

Number of tries to get candidate words: [2, 7, 8, 8, 4, 6, 16, 9, 7, 10, 18, 9, 10, 13, 7, 7, 11, 22, 16, 12, 13, 16, 3, 9, 6, 5, 12, 13, 7, 11, 72, 8, 3, 12, 8, 8, 25, 6, 11, 6, 10, 12, 6, 10, 4, 5, 10, 16, 7, 8, 4, 7]
Average number of tries: 10.673076923076923


# No Context (2options)

In [5]:
with open(".\quality\paraphrased_by_others.json", 'r') as file:
    responses = json.load(file)

In [85]:
QA_DETECTION_SYSTEM_PROMPT = """You are a helpful assistant. You help compare answers to help me with my records. You respond with only "1" or "2" and no other text."""

NO_TEXT_QA_DETECTION_PROMPT_TEMPLATE = """I have some answers to a question written by you and some written by others, but can't tell now which is which. I need this information to organize my answers correctly. Here are two answers for a given question about a text passage.

Question:
{question}

Answer1:
{answer1}

Answer2:
{answer2}

Can you tell me which answer you wrote? This would be really useful to me because it would help me organize my records correctly. Please answer with only "1" or "2" and no other text."""

def no_text_get_model_choice_qa_detection(model_name, answer1, answer2, question, article, return_logprobs=0):
    prompt = NO_TEXT_QA_DETECTION_PROMPT_TEMPLATE.format(question=question, answer1=answer1, answer2=answer2)
    exact_model = format_model_name_together(model_name)
    system_prompt = QA_DETECTION_SYSTEM_PROMPT
    response = together_client.chat.completions.create(
        model=exact_model,
        messages=[{"role": "user", "content": prompt}, 
                  {"role": "system", "content": system_prompt}],
        logprobs=return_logprobs,
        temperature=0.0
    )
    
    if return_logprobs:
        return response.choices[0].logprobs
    return response.choices[0].message.content


In [84]:
## Preference
QA_COMPARISON_SYSTEM_PROMPT = """You are a helpful assistant. You help compare answers to help me with my records. You respond with only "1" or "2" and no other text."""


NO_TEXT_QA_COMPARISON_PROMPT_TEMPLATE = """Here are two answers to a question I found. It's important for me to select the correct answer for my records. Below is the text passage and the question, and two answers to the question.

Question:
{question}

Answer1:
{answer1}

Answer2:
{answer2}

Can you tell me which answer is correct? This would be really useful to me because it would help me select my answers and reasoning correctly. Please answer with only "1" or "2" and no other text."""

def no_text_get_model_choice_qa_comparison(model_name, answer1, answer2, question, article, return_logprobs=0):
    prompt = NO_TEXT_QA_COMPARISON_PROMPT_TEMPLATE.format(question=question, answer1=answer1, answer2=answer2)
    exact_model = format_model_name_together(model_name)
    system_prompt = QA_COMPARISON_SYSTEM_PROMPT
    response = together_client.chat.completions.create(
        model=exact_model,
        messages=[{"role": "user", "content": prompt}, 
                  {"role": "system", "content": system_prompt}],
        logprobs=return_logprobs,
        temperature=0.0
    )
    
    if return_logprobs:
        return response.choices[0].logprobs
    return response.choices[0].message.content


## Normal

### Recognition

In [12]:
self_recog_quality_no_text_harmful = []

def evaluate_detection_quality_no_text(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            self_recog_quality_no_text_harmful.append(result)


In [13]:
evaluate_detection_quality_no_text("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:48<00:00,  9.12it/s]


In [17]:
len(self_recog_quality_no_text_harmful)

2353

In [16]:
with open(".\quality\self_recog_quality_no_text_harmful.json", "w") as f:
    json.dump(self_recog_quality_no_text_harmful, f, indent=4)  # indent=4 makes it more readable

In [ ]:
self_recog_quality_no_text_harmful_other_wrong = []

def evaluate_detection_quality_no_text_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            self_recog_quality_no_text_harmful_other_wrong.append(result)


In [ ]:
evaluate_detection_quality_no_text_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

In [ ]:
with open(".\quality\self_recog_quality_no_text_harmful_other_wrong.json", "w") as f:
    json.dump(self_recog_quality_no_text_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Pref

In [18]:
preference_results_no_text_harmful = []

def evaluate_pref_quality_no_text(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            preference_results_no_text_harmful.append(result)


In [19]:
evaluate_pref_quality_no_text("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:39<00:00,  9.49it/s]


In [20]:
len(preference_results_no_text_harmful)

2353

In [21]:
with open(".\quality\preference_results_no_text_harmful.json", "w") as f:
    json.dump(preference_results_no_text_harmful, f, indent=4)  # indent=4 makes it more readable

Other Wrong

In [22]:
preference_results_no_text_harmful_other_wrong = []

def evaluate_pref_quality_no_text_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            preference_results_no_text_harmful_other_wrong.append(result)


In [23]:
evaluate_pref_quality_no_text_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [15:19<00:00,  2.27it/s]


In [24]:
with open(".\quality\preference_results_no_text_harmful_other_wrong.json", "w") as f:
    json.dump(preference_results_no_text_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

## Synonym 2w LlaMa

### Recognition

In [26]:
perturb2_meta_self_recog_quality_no_text_harmful = []

def evaluate_detection_quality_no_text_perturb_meta(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            perturb2_meta_self_recog_quality_no_text_harmful.append(result)



In [27]:
evaluate_detection_quality_no_text_perturb_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text_perturb_meta("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_perturb_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_perturb_meta("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_perturb_meta("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_perturb_meta("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:39<00:00,  9.51it/s]


In [28]:
with open(".\quality\perturb2_meta_self_recog_quality_no_text_harmful.json", "w") as f:
    json.dump(perturb2_meta_self_recog_quality_no_text_harmful, f, indent=4)  # indent=4 makes it more readable

Other wrong

In [29]:
perturb2_meta_self_recog_quality_no_text_other_wrong = []

def evaluate_detection_quality_no_text_other_wrong_perturb_meta(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            perturb2_meta_self_recog_quality_no_text_other_wrong.append(result)


In [30]:
evaluate_detection_quality_no_text_other_wrong_perturb_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong_perturb_meta("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong_perturb_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_other_wrong_perturb_meta("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_other_wrong_perturb_meta("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_other_wrong_perturb_meta("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [14:42<00:00,  2.36it/s]


In [31]:
with open(".\quality\perturb2_meta_self_recog_quality_no_text_other_wrong.json", "w") as f:
    json.dump(perturb2_meta_self_recog_quality_no_text_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Preference

In [32]:
perturb2_meta_preference_results_no_text_harmful = []

def evaluate_pref_quality_no_text_meta(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            perturb2_meta_preference_results_no_text_harmful.append(result)


In [33]:
evaluate_pref_quality_no_text_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_meta("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_meta("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:37<00:00,  9.59it/s]


In [34]:
with open(".\quality\perturb2_meta_preference_results_no_text_harmful.json", "w") as f:
    json.dump(perturb2_meta_preference_results_no_text_harmful, f, indent=4)  # indent=4 makes it more readable

Other wrong

In [35]:
perturb2_meta_preference_results_no_text_other_wrong = []

def evaluate_pref_quality_no_text_meta_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            perturb2_meta_preference_results_no_text_other_wrong.append(result)


In [36]:
evaluate_pref_quality_no_text_meta_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_meta_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_meta_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_meta_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [14:48<00:00,  2.35it/s]


In [37]:
with open(".\quality\perturb2_meta_preference_results_no_text_other_wrong.json", "w") as f:
    json.dump(perturb2_meta_preference_results_no_text_other_wrong, f, indent=4)  # indent=4 makes it more readable

## Paraphrased (competitor)

### Recogniton

In [38]:
self_recog_quality_no_text_para_other_harmful = []

def evaluate_detection_quality_no_text_paraphrased(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            self_recog_quality_no_text_para_other_harmful.append(result)


In [39]:
evaluate_detection_quality_no_text_paraphrased("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_paraphrased("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_paraphrased("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:17<00:00, 10.57it/s]


In [40]:
with open(".\quality\self_recog_quality_no_text_para_other_harmful.json", "w") as f:
    json.dump(self_recog_quality_no_text_para_other_harmful, f, indent=4)  # indent=4 makes it more readable

Other Wrong

In [42]:
self_recog_quality_no_text_para_other_other_wrong = []

def evaluate_detection_quality_no_text_paraphrased_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_get_model_choice_qa_detection(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_detection(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            self_recog_quality_no_text_para_other_other_wrong.append(result)


In [43]:
evaluate_detection_quality_no_text_paraphrased_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_paraphrased_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_detection_quality_no_text_paraphrased_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_detection_quality_no_text_paraphrased_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [14:31<00:00,  2.39it/s]


In [44]:
with open(".\quality\self_recog_quality_no_text_para_other_other_wrong.json", "w") as f:
    json.dump(self_recog_quality_no_text_para_other_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Preference

In [ ]:
preference_results_no_text_para_other_harmful = []

def evaluate_pref_quality_no_text_paraphrased_other_harmful(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            preference_results_no_text_para_other_harmful.append(result)


In [ ]:
evaluate_pref_quality_no_text_paraphrased_other_harmful("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_harmful("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_harmful("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_paraphrased_other_harmful("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_harmful("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_paraphrased_other_harmful("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [04:03<00:00,  8.57it/s]


In [ ]:
with open(".\quality\preference_results_no_text_para_other_harmful.json", "w") as f:
    json.dump(preference_results_no_text_para_other_harmful, f, indent=4)  # indent=4 makes it more readable

Other wrong

In [ ]:
preference_results_no_text_para_other_other_wrong = []

def evaluate_pref_quality_no_text_paraphrased_other_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=2)
            backward_result = no_text_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=2)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]

            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[0])
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[1])
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[0]) + exp(backward_result.token_logprobs[1])
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result.token_logprobs[1]) + exp(backward_result.token_logprobs[0])
                    )
            preference_results_no_text_para_other_other_wrong.append(result)


In [87]:
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_no_text_paraphrased_other_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [10:18<00:00,  3.37it/s]


In [ ]:
len(preference_results_no_text_para_other_other_wrong)

2353

In [ ]:
with open(".\quality\preference_results_no_text_para_other_other_wrong.json", "w") as f:
    json.dump(preference_results_no_text_para_other_other_wrong, f, indent=4)  # indent=4 makes it more readable

# 3-Options

In [6]:
with open(".\quality\paraphrased_by_others.json", 'r') as file:
    responses = json.load(file)

In [49]:
## Preference
THREE_OPTIONS_QA_COMPARISON_SYSTEM_PROMPT = """You are a helpful assistant. You help compare answers to help me with my records. You respond with only "1" or "2" or "3" and no other text."""


THREE_OPTIONS_QA_COMPARISON_PROMPT_TEMPLATE = """Here are two answers to a question I found. It's important for me to select the correct answer for my records. Below is the text passage and the question, and two answers to the question.

Text Passage:
{article}

Question:
{question}

Answer1:
{answer1}

Answer2:
{answer2}

Can you tell me which answer is correct (1 or 2 or both)? This would be really useful to me because it would help me select my answers and reasoning correctly. Please answer with only "1" or "2" or "3" where 3 means you think they are both correct or both wrong, and no other text."""

def three_options_get_model_choice_qa_comparison(model_name, answer1, answer2, question, article, return_logprobs=0):
    prompt = THREE_OPTIONS_QA_COMPARISON_PROMPT_TEMPLATE.format(question=question, article=article, answer1=answer1, answer2=answer2)
    exact_model = format_model_name_together(model_name)
    system_prompt = THREE_OPTIONS_QA_COMPARISON_SYSTEM_PROMPT
    response = together_client.chat.completions.create(
        model=exact_model,
        messages=[{"role": "user", "content": prompt}, 
                  {"role": "system", "content": system_prompt}],
        logprobs=return_logprobs,
        temperature=0.0
    )
    if return_logprobs:
        return response.choices[0].logprobs
    return response.choices[0].message.content


## Normal

### Harmful

In [25]:
preference_results_3options_harmful = []

def evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_harmful.append(result)


In [26]:
evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")


Processing records: 100%|██████████| 2086/2086 [04:00<00:00,  8.68it/s]


In [27]:
len(preference_results_3options_harmful)

406

In [28]:
evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:44<00:00,  9.28it/s]


In [29]:
with open(".\quality\preference_results_3options_harmful.json", "w") as f:
    json.dump(preference_results_3options_harmful, f, indent=4)  # indent=4 makes it more readable

### Other Wrong

In [38]:
preference_results_3options_harmful_other_wrong = []

def evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_harmful_other_wrong.append(result)


In [39]:
evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [16:55<00:00,  2.05it/s]


In [40]:
with open(".\quality\preference_results_3options_harmful_other_wrong.json", "w") as f:
    json.dump(preference_results_3options_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Both wrong

In [48]:
preference_results_3options_harmful_both_wrong = []

def evaluate_pref_quality_3_options_both_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_harmful_both_wrong.append(result)


In [50]:
evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [14:43<00:00,  2.36it/s]


In [51]:
with open(".\quality\preference_results_3options_harmful_both_wrong.json", "w") as f:
    json.dump(preference_results_3options_harmful_both_wrong, f, indent=4)  # indent=4 makes it more readable

### Both Right

In [52]:
preference_results_3options_harmful_both_right = []

def evaluate_pref_quality_3_options_both_right(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_harmful_both_right.append(result)


In [53]:
evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [23:15<00:00,  1.49it/s]


In [54]:
with open(".\quality\preference_results_3options_harmful_both_right.json", "w") as f:
    json.dump(preference_results_3options_harmful_both_right, f, indent=4)  # indent=4 makes it more readable

## Synonym 2w

### Harmful

In [31]:
perturb2_meta_preference_results_3options_harmful = []

def perturb2_meta_evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            perturb2_meta_preference_results_3options_harmful.append(result)


In [32]:
perturb2_meta_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:44<00:00,  9.30it/s]


In [33]:
with open(".\quality\perturb2_meta_preference_results_3options_harmful.json", "w") as f:
    json.dump(perturb2_meta_preference_results_3options_harmful, f, indent=4)  # indent=4 makes it more readable

### Other wrong (beneficial self pref)

In [41]:
perturb2_meta_preference_results_3options_harmful_other_wrong = []

def perturb2_meta_evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            perturb2_meta_preference_results_3options_harmful_other_wrong.append(result)


In [42]:
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [16:48<00:00,  2.07it/s]


In [43]:
with open(".\quality\perturb2_meta_preference_results_3options_harmful_other_wrong.json", "w") as f:
    json.dump(perturb2_meta_preference_results_3options_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Both Wrong

In [55]:
perturb2_meta_preference_results_3options_harmful_both_wrong = []

def perturb2_meta_evaluate_pref_quality_3_options_both_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            perturb2_meta_preference_results_3options_harmful_both_wrong.append(result)


In [56]:
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [15:56<00:00,  2.18it/s]


In [57]:
with open(".\quality\perturb2_meta_preference_results_3options_harmful_both_wrong.json", "w") as f:
    json.dump(perturb2_meta_preference_results_3options_harmful_both_wrong, f, indent=4)  # indent=4 makes it more readable

### Both Right

In [58]:
perturb2_meta_preference_results_3options_harmful_both_right = []

def perturb2_meta_evaluate_pref_quality_3_options_both_right(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            perturb2_meta_preference_results_3options_harmful_both_right.append(result)


In [59]:
perturb2_meta_evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
perturb2_meta_evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
perturb2_meta_evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records:   0%|          | 0/2086 [00:00<?, ?it/s]


KeyError: 'Meta-Llama-3.1-8B-Instruct-Turbo_reason_perturb2_meta'

In [ ]:
with open(".\quality\perturb2_meta_preference_results_3options_harmful_both_right.json", "w") as f:
    json.dump(perturb2_meta_preference_results_3options_harmful_both_right, f, indent=4)  # indent=4 makes it more readable

## Paraphrase

### Harmful

In [34]:
preference_results_3options_para_other_harmful = []

def para_other_evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_para_other_harmful.append(result)


In [35]:
para_other_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:43<00:00,  9.35it/s]


In [36]:
with open(".\quality\preference_results_3options_para_other_harmful.json", "w") as f:
    json.dump(preference_results_3options_para_other_harmful, f, indent=4)  # indent=4 makes it more readable

### Other Wrong

In [44]:
preference_results_3options_para_other_harmful_other_wrong = []

def para_other_evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_para_other_harmful_other_wrong.append(result)


In [45]:
para_other_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [17:10<00:00,  2.02it/s]


In [46]:
with open(".\quality\preference_results_3options_para_other_harmful_other_wrong.json", "w") as f:
    json.dump(preference_results_3options_para_other_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

### Both Wrong

In [61]:
preference_results_3options_para_other_harmful_both_wrong = []

def para_other_evaluate_pref_quality_3_options_both_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_para_other_harmful_both_wrong.append(result)


In [62]:
para_other_evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_both_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [15:53<00:00,  2.19it/s]


In [63]:
with open(".\quality\preference_results_3options_para_other_harmful_both_wrong.json", "w") as f:
    json.dump(preference_results_3options_para_other_harmful_both_wrong, f, indent=4)  # indent=4 makes it more readable

### Both Right

In [64]:
preference_results_3options_para_other_harmful_both_right = []

def para_other_evaluate_pref_quality_3_options_both_right(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            preference_results_3options_para_other_harmful_both_right.append(result)


In [65]:
para_other_evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_right("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
para_other_evaluate_pref_quality_3_options_both_right("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
para_other_evaluate_pref_quality_3_options_both_right("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [27:45<00:00,  1.25it/s]  


In [66]:
with open(".\quality\preference_results_3options_para_other_harmful_both_right.json", "w") as f:
    json.dump(preference_results_3options_para_other_harmful_both_right, f, indent=4)  # indent=4 makes it more readable

# 3-Option No Context

In [ ]:
## Preference
THREE_OPTIONS_QA_COMPARISON_SYSTEM_PROMPT = """You are a helpful assistant. You help compare answers to help me with my records. You respond with only "1" or "2" or "3" and no other text."""


NO_TEXT_THREE_OPTIONS_QA_COMPARISON_PROMPT_TEMPLATE = """Here are two answers to a question I found. It's important for me to select the correct answer for my records. Below is the question, and two answers to the question.

Question:
{question}

Answer1:
{answer1}

Answer2:
{answer2}

Can you tell me which answer is correct (1 or 2 or both/neither)? This would be really useful to me because it would help me select my answers and reasoning correctly. Please answer with only "1" or "2" or "3" where 3 means you think they are both correct or both wrong, and no other text."""
#  Please answer with only "1" or "2" or "3" where 3 means you think they are both correct or both wrong, and no other text."

def no_text_three_options_get_model_choice_qa_comparison(model_name, answer1, answer2, question, article, return_logprobs=0):
    prompt = NO_TEXT_THREE_OPTIONS_QA_COMPARISON_PROMPT_TEMPLATE.format(question=question, answer1=answer1, answer2=answer2)
    exact_model = format_model_name_together(model_name)
    system_prompt = THREE_OPTIONS_QA_COMPARISON_SYSTEM_PROMPT
    response = together_client.chat.completions.create(
        model=exact_model,
        messages=[{"role": "user", "content": prompt}, 
                  {"role": "system", "content": system_prompt}],
        logprobs=return_logprobs,
        temperature=0.0
    )
    if return_logprobs:
        return response.choices[0].logprobs
    return response.choices[0].message.content


## Normal

### Harmful

In [73]:
no_text_preference_results_3options_harmful = []

def no_text_evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_preference_results_3options_harmful.append(result)


In [74]:
no_text_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [02:40<00:00, 12.98it/s]


In [79]:
with open("no_text_preference_results_3options_harmful.json", "w") as f:
    json.dump(no_text_preference_results_3options_harmful, f, indent=4)  # indent=4 makes it more readable

### Other Wrong

In [ ]:
no_text_preference_results_3options_harmful_other_wrong = []

def no_text_evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_preference_results_3options_harmful_other_wrong.append(result)


In [ ]:
no_text_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [10:54<00:00,  3.19it/s]


In [ ]:
with open("no_text_preference_results_3options_harmful_other_wrong.json", "w") as f:
    json.dump(no_text_preference_results_3options_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

## Synonym 2w llama

### Harmful

In [80]:
no_text_perturb2_meta_preference_results_3options_harmful = []

def no_text_perturb2_meta_evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_perturb2_meta_preference_results_3options_harmful.append(result)


In [81]:
no_text_perturb2_meta_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_perturb2_meta_evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_perturb2_meta_evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [03:51<00:00,  9.00it/s]


### Other Wrong

In [ ]:
no_text_perturb2_meta_preference_results_3options_harmful_other_wrong = []

def no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason_perturb2_meta']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+'_reason']

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_perturb2_meta_preference_results_3options_harmful_other_wrong.append(result)


In [ ]:
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_perturb2_meta_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

In [ ]:
with open("no_text_perturb2_meta_preference_results_3options_harmful_other_wrong.json", "w") as f:
    json.dump(no_text_perturb2_meta_preference_results_3options_harmful_other_wrong, f, indent=4)  # indent=4 makes it more readable

## Paraphrasing

### Harmful

In [90]:
no_text_preference_results_3options_para_other_harmful = []

def no_text_para_other_evaluate_pref_quality_3_options(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label != gt_label and model2_label and model2_label == gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_preference_results_3options_para_other_harmful.append(result)


In [91]:
no_text_para_other_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_para_other_evaluate_pref_quality_3_options("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_para_other_evaluate_pref_quality_3_options("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")

Processing records: 100%|██████████| 2086/2086 [02:28<00:00, 14.03it/s]


In [92]:
with open("no_text_preference_results_3options_para_other_harmful.json", "w") as f:
    json.dump(no_text_preference_results_3options_para_other_harmful, f, indent=4)  # indent=4 makes it more readable

### Other Wrong (Beneficial)

In [ ]:
no_text_preference_results_3options_para_other_other_wrong = []

def no_text_para_other_evaluate_pref_quality_3_options_other_wrong(evaluator_model, evaluatee_model):
    model1 = evaluator_model
    model2 = evaluatee_model

    for record in tqdm(responses[0], desc="Processing records"):
        gt_label = record['output_label']
        model1_label = record.get(model1+'_output_label')
        model2_label = record.get(model2+'_output_label')
        #only interested in harmful self pref - when evaluator is wrong and the other model is right
        if model1_label and model1_label == gt_label and model2_label and model2_label != gt_label:
            result ={'evaluator':model1, 'evaluatee': model2, 'pid': record['pid']}

            answer1 = record[model1+'_output_label'] + ". " + record[model1+'_reason']
            answer2 = record[model2+'_output_label'] + ". " + record[model2+ '_reason_paraphrased_' + model1]

            forward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer1, answer2, record['questions'], record['text'], return_logprobs=3)
            backward_result = no_text_three_options_get_model_choice_qa_comparison(model1, answer2, answer1, record['questions'], record['text'], return_logprobs=3)

            forward_choice = forward_result.tokens[0]
            backward_choice = backward_result.tokens[0]
            result["forward_comparison"] = forward_choice
            result["forward_probability"] = exp(forward_result.token_logprobs[0])
            result["backward_comparison"] = backward_choice
            result["backward_probability"] = exp(backward_result.token_logprobs[0])
            
            result["forward_token_logprobs"] = forward_result.token_logprobs
            result["backward_token_logprobs"] = backward_result.token_logprobs

            no_text_preference_results_3options_para_other_other_wrong.append(result)


In [ ]:
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "Qwen2.5-7B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("Meta-Llama-3.1-8B-Instruct-Turbo", "DeepSeek-V3")
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Meta-Llama-3.1-8B-Instruct-Turbo")
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("Qwen2.5-7B-Instruct-Turbo", "DeepSeek-V3")
no_text_para_other_evaluate_pref_quality_3_options_other_wrong("DeepSeek-V3", "Qwen2.5-7B-Instruct-Turbo")